### Imports and Config

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset, load_metric, Dataset

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

In [2]:
# checkpoint = "bert-base-uncased"
# checkpoint = "roberta-base"
checkpoint = "bert-large-uncased"

### Loading the data

In [3]:
raw_dataset = load_dataset("my_dataset_loading_script", "condemnation")

Reusing dataset condemnation_dataset (/home/geev/.cache/huggingface/datasets/condemnation_dataset/condemnation/1.1.0/ef891eb2986445dbe69883df48d98d7039da591e0c35cb211978c71951a3c83e)


In [4]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1344
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 336
    })
})

In [5]:
raw_train_dataset = raw_dataset["train"]
raw_train_dataset[0]

{'text': 'As multiple sexual allegations come to light with people like <OTHER TARGET 1>, NPR’s -JOHN DOE-, and Michael Fallon, just to name a few...',
 'label': 1}

In [6]:
raw_train_dataset.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['no_condemnation', 'condemnation'], names_file=None, id=None)}

### Preprocessing & Evaluation

In [7]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at /home/geev/.cache/huggingface/datasets/condemnation_dataset/condemnation/1.1.0/ef891eb2986445dbe69883df48d98d7039da591e0c35cb211978c71951a3c83e/cache-d225f7e2b30c3b0a.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
def evaluate(trainer, tokenized_dataset):
    predictions = trainer.predict(tokenized_dataset)
    preds = np.argmax(predictions.predictions, axis=1)
    def get_metrics(y_pred, y_true):
        metrics ={}
        metrics["accuracy"] = accuracy_score(y_true, y_pred)
        metrics["precision"] = precision_score(y_true, y_pred)
        metrics["recall"] = recall_score(y_true, y_pred)
        metrics["f1"] = f1_score(y_true, y_pred)
        return metrics
    return get_metrics(preds, predictions.label_ids)


### Training with CV

In [9]:
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]
n_split = 10
sss = StratifiedShuffleSplit(n_splits=n_split, test_size=1/n_split, random_state=0)
sss.get_n_splits(train_dataset, train_dataset["label"])

10

In [10]:
torch.cuda.empty_cache()

In [11]:
performance = {"test":{}, "val":{}}
fold = 1
for train_index, val_index in sss.split(train_dataset, train_dataset["label"]):
    cv_train_dataset = Dataset.from_dict(train_dataset[train_index])
    
    training_args = TrainingArguments("test_trainer")
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
    
    trainer = Trainer(
    model,
    training_args,
    train_dataset = cv_train_dataset,
    data_collator = data_collator,
    tokenizer = tokenizer,
    )
    
    trainer.train()
    cv_val_dataset = Dataset.from_dict(train_dataset[val_index])

    performance["val"]["fold "+str(fold)] = evaluate(trainer, cv_val_dataset)
    performance["test"]["fold "+str(fold)] = evaluate(trainer, test_dataset)
    fold+=1
    del model
    del trainer
    del cv_train_dataset
    del cv_val_dataset
    torch.cuda.empty_cache()
    
    

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

Step,Training Loss


RuntimeError: CUDA out of memory. Tried to allocate 120.00 MiB (GPU 0; 7.80 GiB total capacity; 6.14 GiB already allocated; 97.56 MiB free; 6.41 GiB reserved in total by PyTorch)

In [ ]:
import pickle
pickle_dump_file = "10_cv_"+checkpoint+"_performance.p"
pickle.dump(performance, open(pickle_dump_file, "wb"))

In [ ]:
pickle.load(open(pickle_dump_file, "rb"))

In [ ]:
def agg_performance(performance):
    
    agg_performance = {}
    for key in performance:
        agg_performance[key] = {}
        for metric in performance[key]["fold 1"]:
            metric_val_list = [performance[key][fold][metric] for fold in performance[key]]
            agg_performance[key]["avg_"+metric] = "{:.2f}".format(np.mean(metric_val_list))
            agg_performance[key]["std_"+metric] = "{:.2f}".format(np.std(metric_val_list))
    return agg_performance
        
import pprint
pprint.pprint(agg_performance(performance)        )